## Set how many days to back up

In [1]:
NoDaysInThePast = 1

StatementMeta(, 648301f7-29f6-473a-82db-15b402825325, 3, Finished, Available, Finished)

## Import required libraries

In [2]:
#Import necessary libraries
import msal
import requests
import json
import pandas as pd
from datetime import date, timedelta

StatementMeta(, 648301f7-29f6-473a-82db-15b402825325, 4, Finished, Available, Finished)

## Get the date depending on how many days to back up and convert to string

In [3]:

activityDate = date.today() - timedelta(days=NoDaysInThePast)
activityDate = activityDate.strftime("%Y-%m-%d")

StatementMeta(, 648301f7-29f6-473a-82db-15b402825325, 5, Finished, Available, Finished)

## Set the client ID and Secret for the Service principal which has access

In [4]:

client_id = "XXXXXXXX-1234-1234-1234-XXXXXXXXXXXX"
client_secret = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
authority_url = "https://login.microsoftonline.com/northinsights.com"
scope = ["https://analysis.windows.net/powerbi/api/.default"]

StatementMeta(, 648301f7-29f6-473a-82db-15b402825325, 6, Finished, Available, Finished)

## Set Power BI REST API to get Activities for the correct day

In [5]:

url = "https://api.powerbi.com/v1.0/myorg/admin/activityevents?startDateTime='" + activityDate + "T00:00:00'&endDateTime='" + activityDate + "T23:59:59'"

StatementMeta(, 648301f7-29f6-473a-82db-15b402825325, 7, Finished, Available, Finished)

## Set the path where the JSON files should be saved

In [6]:

path = 'abfss://221e71a3-3156-4334-8c4c-9715a43e871f@onelake.dfs.fabric.microsoft.com/fcb3130d-16f6-419a-9c16-acd8ca3abc49/Files/pbiactivitylog'

StatementMeta(, 648301f7-29f6-473a-82db-15b402825325, 8, Finished, Available, Finished)

## Use MSAL to fetch a token for authentication

In [7]:

app = msal.ConfidentialClientApplication(client_id, authority=authority_url, client_credential=client_secret)
result = app.acquire_token_for_client(scopes=scope)

StatementMeta(, 648301f7-29f6-473a-82db-15b402825325, 9, Finished, Available, Finished)

## Use MSAL to grab token
### Then get the activities via the REST API
### If there are more than one page (hour) of data continue until we are done

In [8]:

app = msal.ConfidentialClientApplication(client_id, authority=authority_url, client_credential=client_secret)
result = app.acquire_token_for_client(scopes=scope)
#Get latest Power BI Activities
if 'access_token' in result:
    access_token = result['access_token']
    header = {'Content-Type':'application/json', 'Authorization':f'Bearer {access_token}'}
    api_call = requests.get(url=url, headers=header)
json_rdd = spark.sparkContext.parallelize([api_call.text])
raw_df = spark.read.json(json_rdd)
display(raw_df)


contUrl = api_call.json()['continuationUri']
print(contUrl)

while contUrl is not None:        
    api_call_cont = requests.get(url=contUrl, headers=header)
    contUrl = api_call_cont.json()['continuationUri']
    json_rdd2 = spark.sparkContext.parallelize([api_call_cont.text])
    raw_df2 = spark.read.json(json_rdd2)
    raw_df = raw_df.unionByName(raw_df2, allowMissingColumns=True)
display(raw_df)

StatementMeta(, 648301f7-29f6-473a-82db-15b402825325, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7d8536b9-9bea-49c1-bb08-139e44f3cdfe)

https://wabi-europe-north-b-redirect.analysis.windows.net/v1.0/myorg/admin/activityevents?continuationToken='eyJTdGFydERhdGVUaW1lIjoiMjAyNC0xMC0wM1QwMDowMDowMC4wMDAwMDAwXHUwMDJCMDA6MDAiLCJFbmREYXRlVGltZSI6IjIwMjQtMTAtMDNUMjM6NTk6NTkuMDAwMDAwMFx1MDAyQjAwOjAwIiwiRmlsZU5hbWUiOiIyMDI0LTEwLTAzVDAxX3YxXzAwMS5jc3YiLCJGaWxlT2Zmc2V0IjowLCJBY3Rpdml0eSI6bnVsbCwiVXNlcklkIjpudWxsfSwyMDI0LTEwLTAzVDAwOjAwOjAwLjAwMDAwMDArMDA6MDAsMjAyNC0xMC0wM1QyMzo1OTo1OS4wMDAwMDAwKzAwOjAwLDAsLA%3D%3D'


SynapseWidget(Synapse.DataFrame, 71401adb-8399-4718-a638-e4d3f0993882)

## Set today as a variable to name the folder to save the JSON files

In [9]:
import datetime
activityDate = str((date.today() - timedelta(days=NoDaysInThePast)).strftime("%Y%m%d"))
activityYear = str((date.today() - timedelta(days=NoDaysInThePast)).strftime("%Y"))
activityMonth = str((date.today() - timedelta(days=NoDaysInThePast)).strftime("%Y%m"))
print(activityMonth)

StatementMeta(, 648301f7-29f6-473a-82db-15b402825325, 11, Finished, Available, Finished)

202410


## Save the JSON files to the Files section of the Lakehouse

In [10]:

#Save df as JSON
tableName = "Files/pbi_activitylog/year=" + activityYear + "/month=" + activityMonth + "/day=" + activityDate + "/pbiactivitylog_" + activityDate
raw_df.write.format('json').option('inferSchema','true').mode('overwrite').save(tableName + '.json')

StatementMeta(, 648301f7-29f6-473a-82db-15b402825325, 12, Finished, Available, Finished)